In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

def resposta_simples(query):
    return f"A resposta para '{query}' é Paris."

tools = [
    Tool(
        name="Respostas Simples",
        func=resposta_simples,
        description="Uma ferramenta simples para responder perguntas diretas"
    )
]

agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

input_query = "QUal é a capital da França?"
response = agent.run(input_query)
print(response)

In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
import sqlite3

def create_connection():
    conn = sqlite3.connect(':memory:')  # Use an in-memory database for simplicity
    c = conn.cursor()
    # Users: id, name, age
    c.execute('''CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')
    # Insert fake data
    c.execute("INSERT INTO users (name, age) VALUES ('Alice', 30)")
    c.execute("INSERT INTO users (name, age) VALUES ('Bob', 25)")
    c.execute("INSERT INTO users (name, age) VALUES ('Charlie', 35)")
    conn.commit()
    return conn

def search_user(name: str):
    conn = create_connection()
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE name=?", (name,))
    result = c.fetchone()
    c.close()
    if result:
        return f"User found: ID={result[0]}, Name={result[1]}, Age={result[2]}"
    else:
        return "User not found."

def sum_numbers(numbers) -> str:
    if isinstance(numbers, str):
        numbers = eval(numbers)
    if isinstance(numbers, list):
        return f"A soma dos números é: {sum(numbers)}"
    return "Entrada inválida para soma."


tools = [
    Tool(
        name="search_user",
        func=search_user,
        description="Search for a user by name in the database. Returns user details if found, otherwise indicates not found."
    ),
    Tool(
        name="sum_numbers",
        func=sum_numbers,
        description="Sums a list of numbers. Input should be a list of numbers."
    )
]

llm = ChatOpenAI(model="gpt-3.5-turbo-0125") # type: ignore
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
input_query = "Qual é a soma de [1, 2, 3, 4] e quem é o usuário Bob?"
response = agent.run(input_query)
print(response)